## Model Registry

In [ ]:
from focoos.model_registry import ModelRegistry

registry = ModelRegistry()
print(registry.list_models())


model_info = registry.get_model_info("fai-detr-l-obj365")

model_info.pprint()

## AutoDataset

In [ ]:
from focoos.data.auto_dataset import AutoDataset
from focoos.data.default_aug import get_default_by_task
from focoos.ports import DatasetLayout, DatasetSplitType, Task

task = Task.DETECTION
layout = DatasetLayout.ROBOFLOW_COCO
auto_dataset = AutoDataset(dataset_name="aquarium", task=task, layout=layout, datasets_dir="../datasets")

train_augs, val_augs = get_default_by_task(task, 640, advanced=False)
train_dataset = auto_dataset.get_split(augs=train_augs.get_augmentations(), split=DatasetSplitType.TRAIN)
valid_dataset = auto_dataset.get_split(augs=val_augs.get_augmentations(), split=DatasetSplitType.VAL)

## Auto Model

In [ ]:
from focoos.model_manager import ModelManager

model = ModelManager.get("fai-detr-m-coco", num_classes=train_dataset.dataset.metadata.num_classes)

## Train with a dataset downloaded from HUB

In [ ]:
from focoos import FocoosHUB
from focoos.data.auto_dataset import AutoDataset
from focoos.data.default_aug import get_default_by_task
from focoos.model_manager import ModelManager
from focoos.ports import DatasetLayout, DatasetSplitType, Task, TrainerArgs

focoos = FocoosHUB()
my_datasets = focoos.list_remote_datasets(include_shared=False)
remote_dataset = focoos.get_remote_dataset(my_datasets[0].ref)
dataset_path = remote_dataset.download_data()
auto_dataset = AutoDataset(dataset_name=dataset_path, task=remote_dataset.task, layout=remote_dataset.layout)

train_augs, val_augs = get_default_by_task(remote_dataset.task, 640, advanced=False)
train_dataset = auto_dataset.get_split(augs=train_augs.get_augmentations(), split=DatasetSplitType.TRAIN)
valid_dataset = auto_dataset.get_split(augs=val_augs.get_augmentations(), split=DatasetSplitType.VAL)


model = ModelManager.get("fai-detr-m-coco", num_classes=train_dataset.dataset.metadata.num_classes)

args = TrainerArgs(
    run_name="footballxyz",
    output_dir="./experiments",
    amp_enabled=True,
    batch_size=16,
    max_iters=50,
    eval_period=100,
    learning_rate=0.0001,
    scheduler="MULTISTEP",
    weight_decay=0.0001,
    workers=16,
)


model.train(args, train_dataset, valid_dataset)
infer = model.export(format="torchscript")
infer.benchmark()

## Validation only

In [ ]:
from focoos.ports import TrainerArgs

args = TrainerArgs(
    run_name="aquarium2",
    output_dir="./experiments",
    amp_enabled=True,
    batch_size=16,
    max_iters=300,
    eval_period=100,
    learning_rate=0.0001,
    scheduler="MULTISTEP",
    weight_decay=0.0001,
    workers=16,
)

model.test(args, valid_dataset)

# Detection

In [ ]:
# Use the model for inference
from PIL import Image

from focoos.data.auto_dataset import AutoDataset
from focoos.data.default_aug import get_default_by_task
from focoos.model_manager import ModelManager
from focoos.models.fai_detr.processor import DETRProcessor
from focoos.ports import DatasetLayout, DatasetSplitType, Task, TrainerArgs

task = Task.DETECTION
layout = DatasetLayout.ROBOFLOW_COCO
auto_dataset = AutoDataset(dataset_name="aquarium", task=task, layout=layout, datasets_dir="../datasets")
resolution = 640

train_augs, val_augs = get_default_by_task(task, resolution, advanced=False)
train_dataset = auto_dataset.get_split(augs=train_augs.get_augmentations(), split=DatasetSplitType.TRAIN)
valid_dataset = auto_dataset.get_split(augs=val_augs.get_augmentations(), split=DatasetSplitType.VAL)


model = ModelManager.get("fai-detr-m-coco", num_classes=train_dataset.dataset.metadata.num_classes)

args = TrainerArgs(
    run_name="footballxyz",
    output_dir="./experiments",
    amp_enabled=True,
    batch_size=16,
    max_iters=50,
    eval_period=100,
    learning_rate=0.0001,
    scheduler="MULTISTEP",
    weight_decay=0.0001,
    workers=16,
)

model.train(args, train_dataset, valid_dataset)

image = Image.open("image.jpg")
postprocessor = DETRProcessor(model.model_info.config)
outputs = postprocessor.postprocess(model(image), image)

print(outputs)

## Image Classification

In [ ]:
# Use the model for inference
from PIL import Image

from focoos.data.auto_dataset import AutoDataset
from focoos.data.default_aug import get_default_by_task
from focoos.model_manager import ConfigManager, ModelManager
from focoos.nn.backbone.resnet import ResnetConfig
from focoos.ports import (
    DatasetLayout,
    DatasetSplitType,
    ModelFamily,
    ModelInfo,
    Task,
    TrainerArgs,
)

task = Task.CLASSIFICATION
layout = DatasetLayout.CLS_FOLDER
auto_dataset = AutoDataset(dataset_name="hymenoptera", task=task, layout=layout, datasets_dir="../datasets")
resolution = 224

train_augs, val_augs = get_default_by_task(task, resolution, advanced=False)
train_dataset = auto_dataset.get_split(augs=train_augs.get_augmentations(), split=DatasetSplitType.TRAIN)
valid_dataset = auto_dataset.get_split(augs=val_augs.get_augmentations(), split=DatasetSplitType.VAL)


# Create a configuration with a ResNet backbone
cls_config = ConfigManager.from_dict(
    ModelFamily.IMAGE_CLASSIFIER,
    {
        "backbone_config": dict(ResnetConfig(model_type="resnet", depth=50, pretrained=True)),
        "num_classes": valid_dataset.dataset.metadata.num_classes,
        "resolution": resolution,
        "hidden_dim": 512,
        "dropout_rate": 0.2,
    },
)

model_info = ModelInfo(
    name="fai-cls-resnet50",
    description="ResNet50 model for classification",
    task=Task.CLASSIFICATION,
    classes=["cat", "dog", "bird"],
    im_size=224,
    model_family=ModelFamily.CLS,
    config=cls_config,
)
# Create the model
model = ModelManager.get(name=model_info.name, model_info=model_info)

args = TrainerArgs(
    run_name="footballxyz",
    output_dir="./experiments",
    amp_enabled=True,
    batch_size=16,
    max_iters=50,
    eval_period=100,
    learning_rate=0.0001,
    scheduler="MULTISTEP",
    weight_decay=0.0001,
    workers=16,
)

model.train(args, train_dataset, valid_dataset)

image = Image.open("image.jpg")
outputs = model(image)

print(outputs)

# SEGMENTATION

In [ ]:
from PIL import Image

from focoos.data.auto_dataset import AutoDataset
from focoos.data.default_aug import get_default_by_task
from focoos.model_manager import ModelManager
from focoos.models.fai_mf.processor import MaskFormerProcessor
from focoos.ports import DatasetLayout, DatasetSplitType, Task, TrainerArgs

task = Task.SEMSEG
layout = DatasetLayout.ROBOFLOW_SEG
auto_dataset = AutoDataset(dataset_name="pizza", task=task, layout=layout, datasets_dir="../datasets")

train_augs, val_augs = get_default_by_task(task, 640, advanced=False)
train_dataset = auto_dataset.get_split(augs=train_augs.get_augmentations(), split=DatasetSplitType.TRAIN)
valid_dataset = auto_dataset.get_split(augs=val_augs.get_augmentations(), split=DatasetSplitType.VAL)

model = ModelManager.get(
    "fai-mf-m-ade"  # , num_classes=valid_dataset.dataset.metadata.num_classes
)

postprocessor = MaskFormerProcessor(config=model.model_info.config)

args = TrainerArgs(
    run_name="footballxyz",
    output_dir="./experiments",
    amp_enabled=True,
    batch_size=16,
    max_iters=50,
    eval_period=100,
    learning_rate=0.0001,
    scheduler="MULTISTEP",
    weight_decay=0.0001,
    workers=16,
)

# model.train(args, train_dataset, valid_dataset)

image = Image.open("image.jpg")
outputs = postprocessor.postprocess(
    model(image),
    image,
    predict_all_pixels=True,
    use_mask_score=False,
    filter_empty_masks=False,
    threshold=0.5,
)

# print(outputs.logits.shape,outputs.masks.shape)
for det in outputs[0].detections:
    print(det.cls_id, det.conf)

In [ ]:
from PIL import Image

from focoos.data.auto_dataset import AutoDataset
from focoos.data.default_aug import get_default_by_task
from focoos.models.fai_mf.processor import MaskFormerProcessor
from focoos.ports import DatasetLayout, DatasetSplitType, Task, TrainerArgs

task = Task.INSTANCE_SEGMENTATION
layout = DatasetLayout.ROBOFLOW_COCO
auto_dataset = AutoDataset(dataset_name="fruits", task=task, layout=layout, datasets_dir="../datasets")

train_augs, val_augs = get_default_by_task(task, 640, advanced=False)
train_dataset = auto_dataset.get_split(augs=train_augs.get_augmentations(), split=DatasetSplitType.TRAIN)
valid_dataset = auto_dataset.get_split(augs=val_augs.get_augmentations(), split=DatasetSplitType.VAL)

model = ModelManager.get(
    "fai-mf-s-coco-ins"  # , num_classes=valid_dataset.dataset.metadata.num_classes
)
postprocessor = MaskFormerProcessor(model.config)

args = TrainerArgs(
    run_name="footballxyz",
    output_dir="./experiments",
    amp_enabled=True,
    batch_size=16,
    max_iters=50,
    eval_period=100,
    learning_rate=0.0001,
    scheduler="MULTISTEP",
    weight_decay=0.0001,
    workers=16,
)

# model.train(args, train_dataset, valid_dataset)

image = Image.open("image.jpg")
outputs = postprocessor.postprocess(model(image), image, use_mask_score=False, filter_empty_masks=True, threshold=0.5)

# print(outputs.logits.shape,outputs.masks.shape)
for det in outputs[0].detections:
    print(det.cls_id, det.bbox, det.conf)

In [ ]:
from focoos.utils.system import get_system_info

get_system_info().pprint()

In [ ]:
from focoos.model_registry import ModelRegistry

registry = ModelRegistry()
print(registry.list_models())

In [ ]:
from focoos.model_manager import ModelManager

model = ModelManager.get("fai-detr-l-obj365")

## Export

In [ ]:
from PIL import Image

from focoos.model_manager import ModelManager
from focoos.model_registry import ModelRegistry

registry = ModelRegistry()

image = Image.open("image.jpg")


# model_name = "fai-detr-n-coco"
print("=============ONNX=============")
for model_name in ["bisenetformer-m-ade"]:
    model = ModelManager.get(model_name)
    infer = model.export(format="onnx", overwrite=True)
    detections, preview = infer.infer(image, annotate=True)
    print(detections)
Image.fromarray(preview)

# print("=============TORCHSCRIPT=============")
# for model_name in registry.list_models():
#     model = ModelManager.get(model_name)
#     infer = model.export(format="torchscript", overwrite=True)
#     detections, preview = infer.infer(image, annotate=True)
#     print(detections)
#     Image.fromarray(preview)